Midterm Project by Gabriel Jackson (tbp8gx)

I focused this project on retail sales of *adventureworks* with the following tables:
1. dim_customer for Customers (via SQL)
2. dim_product for Products (via JSON & MongoDB)
3. dim_date for Dates (via SQL)
4. dim_territory for Territories (via local CSV file)
5. fact_sale_orders for Sales (via SQL)

Project Outline:
1. Import libraries, connect to MongoDB and MySQL, and create helper functions to communicate with MongoDB and MySQL (like in the previous projects)
2. Extract the dimensional dataframes for the dimensional (dim) tables
3. Transform the dimensional dataframes 
4. Load the dimensional dataframes into the new database
5. Extract the fact dataframe via SQL
6. Transform the fact dataframe via SQL
7. Load the fact dataframe into the new database
8. Demonstrate proper functionality in new database


Step 1 - Import libraries, connect to MongoDB and MySQL, and create helper functions

In [1]:
# Step 1.1 - Import libraries
import os
import json
import numpy
import datetime
import certifi
import pandas as pd

import pymongo
import sqlalchemy
from sqlalchemy import create_engine, text

# Step 1.2 - Connect to MongoDB and MySQL
src_db_args = {
    "uid" : "root",
    "pwd" : "password",
    "hostname" : "localhost",
    "dbname" : "adventureworks"
}
dest_db_args = {
    "uid" : "root",
    "pwd" : "password",
    "hostname" : "localhost",
    "dbname" : "midterm_dw"
}
mongodb_args = {
    "user_name" : "Cluster94547",
    "password" : "cWdKdENBS2pj",
    "cluster_name" : "Cluster94547",
    "cluster_subnet" : "hp0v9",
    "cluster_location" : "atlas", 
    "db_name" : "midterm"
}

# Step 1.3 - Create helper function to communicate ith MongoDB and MySQL
def get_sql_dataframe(sql_query, **args):
    '''Create a connection to the MySQL database'''
    conn_str = f"mysql+pymysql://{args['uid']}:{args['pwd']}@{args['hostname']}/{args['dbname']}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()

    '''Invoke the pd.read_sql() function to query the database, and fill a Pandas DataFrame.'''
    dframe = pd.read_sql(text(sql_query), connection);
    connection.close()

    return dframe


def set_dataframe(df, table_name, pk_column, db_operation, **args):
    '''Create a connection to the MySQL database'''
    conn_str = f"mysql+pymysql://{args['uid']}:{args['pwd']}@{args['hostname']}/{args['dbname']}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()

    '''Invoke the Pandas DataFrame .to_sql( ) function to either create, or append to, a table'''
    if db_operation == "insert":
        df.to_sql(table_name, con=connection, index=False, if_exists='replace')
        connection.execute(text(f"ALTER TABLE {table_name} ADD PRIMARY KEY ({pk_column});"))

    elif db_operation == "update":
        df.to_sql(table_name, con=connection, index=False, if_exists='append')

    connection.close()


def get_mongo_client(**args):
    '''Validate proper input'''
    if args["cluster_location"] not in ['atlas', 'local']:
        raise Exception("You must specify either 'atlas' or 'local' for the cluster_location parameter.")

    else:
        if args["cluster_location"] == "atlas":
            connect_str = f"mongodb+srv://{args['user_name']}:{args['password']}@"
            connect_str += f"{args['cluster_name']}.{args['cluster_subnet']}.mongodb.net"
            client = pymongo.MongoClient(connect_str, tlsCAFile=certifi.where())

        elif args["cluster_location"] == "local":
            client = pymongo.MongoClient("mongodb://localhost:27017/")

    return client


def get_mongo_dataframe(mongo_client, db_name, collection, query):
    '''Query MongoDB, and fill a python list with documents to create a DataFrame'''
    db = mongo_client[db_name]
    dframe = pd.DataFrame(list(db[collection].find(query)))
    dframe.drop(['_id'], axis=1, inplace=True)
    mongo_client.close()

    return dframe


def set_mongo_collections(mongo_client, db_name, data_directory, json_files):
    db = mongo_client[db_name]

    for file in json_files:
        db.drop_collection(file)
        json_file = os.path.join(data_directory, json_files[file])
        with open(json_file, 'r') as openfile:
            json_object = json.load(openfile)
            file = db[file]
            result = file.insert_many(json_object)

    mongo_client.close()

In [2]:
# Step 1.4 - Populate MongoDB with dim_products source data (.JSON file in data/products.json file)
client = get_mongo_client(**mongodb_args)
data_dir = os.path.join(os.getcwd(), 'data')

json_files = {
    "products" : 'products.json'
}
set_mongo_collections(client, mongodb_args["db_name"], data_dir, json_files)

Step 2 - Extract the dataframes for the following tables (according to their source):
1. 'customer' -> 'df_dim_customer' (via SQL)
2. 'product' -> 'df_dim_product' (via JSON & MongoDB)
3. 'dim_date' -> 'df_dim_date' (via SQL)
4. 'territory' -> 'df_dim_territory' (via local CSV file)
5. 'salesorders' -> 'df_fact_sales_orders' (via SQL)

In [3]:
# Step 2.1 - Extract dimensional customer data
sql_customer = """
    SELECT 
        c.CustomerID,
        contact.FirstName,
        contact.LastName,
        contact.EmailAddress,
        contact.Phone,
        c.TerritoryID,
        c.AccountNumber,
        c.CustomerType,
        a.AddressLine1,
        a.AddressLine2,
        a.City,
        a.PostalCode,
        sp.Name AS `StateProvince`,
        cr.Name AS `Country`
    FROM adventureworks.customer AS c
    LEFT OUTER JOIN customeraddress AS ca
    ON c.CustomerID = ca.CustomerID
    LEFT OUTER JOIN address AS a 
    ON ca.AddressID = a.AddressID
    LEFT OUTER JOIN addresstype AS t
	ON ca.AddressTypeID = t.AddressTypeID
	LEFT OUTER JOIN stateprovince AS sp
	ON sp.StateProvinceID = a.StateProvinceID
	LEFT OUTER JOIN countryregion AS cr
	ON sp.CountryRegionCode = cr.CountryRegionCode
	LEFT OUTER JOIN individual AS i
	ON c.CustomerID = i.CustomerID
	LEFT OUTER JOIN contact
	ON i.ContactID = contact.ContactID;
"""
df_dim_customers = get_sql_dataframe(sql_customer, **src_db_args)
df_dim_customers.head(2)

,CustomerID,FirstName,LastName,EmailAddress,Phone,TerritoryID,AccountNumber,CustomerType,AddressLine1,AddressLine2,City,PostalCode,StateProvince,Country
0,1,None,None,None,None,1,AW00000001,S,2251 Elliot Avenue,None,Seattle,98104,Washington,United States
1,2,None,None,None,None,1,AW00000002,S,7943 Walnut Ave,None,Renton,98055,Washington,United States


In [4]:
# Step 2.2 - Extract dimensional dim_date data
# NOTE: The 'Lab_02c_Create_Populate_Dim_Date' script was ran on the 'adventureworks' database in order to create the table.
# NOTE: If you wish to replicate these steps, then this must be done on the table.

sql_dim_date = "SELECT date_key, full_date FROM adventureworks.dim_date;"
df_dim_dates = get_sql_dataframe(sql_dim_date, **src_db_args)
df_dim_dates.full_date = df_dim_dates.full_date.astype('datetime64[ns]').dt.date
df_dim_dates.head(2)

,date_key,full_date
0,20000101,2000-01-01
1,20000102,2000-01-02


In [5]:
# Step 2.3 - Extract dimensional product data (.JSON) from MongoDB

client = get_mongo_client(**mongodb_args)
query = {} 
collection = "products"

df_dim_products = get_mongo_dataframe(client, mongodb_args["db_name"], collection, query)
df_dim_products.head(2)

,ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,...,DaysToManufacture,ProductLine,Class,Style,ProductCategory,ProductSubcategory,ProductModel,SellStartDate,SellEndDate,DiscontinuedDate
0,1,Adjustable Race,AR-5381,0,0,None,1000,750,0.0,0.0,...,0,None,None,None,None,None,None,1998-06-01 00:00:00,None,None
1,2,Bearing Ball,BA-8327,0,0,None,1000,750,0.0,0.0,...,0,None,None,None,None,None,None,1998-06-01 00:00:00,None,None


In [6]:
# Step 2.4 - Extract dimensional territory data (.CSV) from local files in data/territory.csv file
df_dim_territories = pd.read_csv('data/territory.csv')
df_dim_territories.head(2)

,TerritoryID,TerritoryName,CountryRegionCode,Group
0,1,Northwest,US,North America
1,2,Northeast,US,North America


Step 3 - Transform the dataframes into dimensional dataframes

In [7]:
# Step 3.1 - Transform dimensional customer table 

# Remove any unwanted data (territory ID should be found in dim_product)
df_dim_customers.drop('TerritoryID', axis=1, inplace=True)

# Add new primary key to dimension table
df_dim_customers.insert(0, "Customer_Key", range(1, df_dim_customers.shape[0]+1))

df_dim_customers.head(2)

,Customer_Key,CustomerID,FirstName,LastName,EmailAddress,Phone,AccountNumber,CustomerType,AddressLine1,AddressLine2,City,PostalCode,StateProvince,Country
0,1,1,None,None,None,None,AW00000001,S,2251 Elliot Avenue,None,Seattle,98104,Washington,United States
1,2,2,None,None,None,None,AW00000002,S,7943 Walnut Ave,None,Renton,98055,Washington,United States


In [8]:
# Step 3.2 - Transform dimensional product table
# NOTE: A lot of the initial rows do no contain full data (as they are old), so the head(2) results are not representative of the full dataset.

# Remove any unwanted data
df_dim_products.drop('Style', axis=1, inplace=True)
df_dim_products.drop('ProductLine', axis=1, inplace=True)
df_dim_products.drop('Class', axis=1, inplace=True)
df_dim_products.drop('SellStartDate', axis=1, inplace=True)
df_dim_products.drop('SellEndDate', axis=1, inplace=True)
df_dim_products.drop('DiscontinuedDate', axis=1, inplace=True)

# Change flags to booleans 
df_dim_products['MakeFlag'] = df_dim_products.MakeFlag.map({"1" :'True', "0" :'False'})
df_dim_products['FinishedGoodsFlag'] = df_dim_products.FinishedGoodsFlag.map({"1" :'True', "0" :'False'})

# Add new primary key to dimension table
df_dim_products.insert(0, "Product_Key", range(1, df_dim_products.shape[0]+1))

df_dim_products.head(2)

,Product_Key,ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Size,SizeUnitMeasureCode,WeightUnitMeasureCode,Weight,DaysToManufacture,ProductCategory,ProductSubcategory,ProductModel
0,1,1,Adjustable Race,AR-5381,False,False,None,1000,750,0.0,0.0,None,None,None,NaN,0,None,None,None
1,2,2,Bearing Ball,BA-8327,False,False,None,1000,750,0.0,0.0,None,None,None,NaN,0,None,None,None


In [9]:
# Step 3.3 - Transform dimensional territory table

# Add new primary key to dimension table
df_dim_territories.insert(0, "Territory_Key", range(1, df_dim_territories.shape[0]+1))

df_dim_territories.head(2)

,Territory_Key,TerritoryID,TerritoryName,CountryRegionCode,Group
0,1,1,Northwest,US,North America
1,2,2,Northeast,US,North America


In [10]:
# Step 4.1 - Load dimensional tables into database
# set_dataframe(df_dim_customers, "dim_products", "Product_key", "insert", **dest_db_args)
set_dataframe(df_dim_dates, "dim_date", "date_key", "insert", **dest_db_args)
set_dataframe(df_dim_customers, "dim_customers", "Customer_key", "insert", **dest_db_args)
set_dataframe(df_dim_territories, "dim_territories", "Territory_key", "insert", **dest_db_args)

In [11]:
# Step 4.2 - Validate loading of dim_products table into database
sql_products = "SELECT * FROM midterm_dw.dim_products;"
df_dim_products = get_sql_dataframe(sql_products, **dest_db_args)
df_dim_products.head(2)

,Product_Key,ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Size,SizeUnitMeasureCode,WeightUnitMeasureCode,Weight,DaysToManufacture,ProductCategory,ProductSubcategory,ProductModel
0,1,1,Adjustable Race,AR-5381,False,False,None,1000,750,0.0,0.0,None,None,None,NaN,0,None,None,None
1,2,2,Bearing Ball,BA-8327,False,False,None,1000,750,0.0,0.0,None,None,None,NaN,0,None,None,None


In [12]:
# Step 4.3 - Validate loading of dim_date table into database
sql_date = "SELECT * FROM midterm_dw.dim_date;"
df_dim_dates = get_sql_dataframe(sql_date, **dest_db_args)
df_dim_dates.head(2)

,date_key,full_date
0,20000101,2000-01-01
1,20000102,2000-01-02


In [13]:
# Step 4.4 - Validate loading of dim_customers table into database
sql_customers = "SELECT * FROM midterm_dw.dim_customers;"
df_dim_customers = get_sql_dataframe(sql_customers, **dest_db_args)
df_dim_customers.head(2)

,Customer_Key,CustomerID,FirstName,LastName,EmailAddress,Phone,AccountNumber,CustomerType,AddressLine1,AddressLine2,City,PostalCode,StateProvince,Country
0,1,1,None,None,None,None,AW00000001,S,2251 Elliot Avenue,None,Seattle,98104,Washington,United States
1,2,2,None,None,None,None,AW00000002,S,7943 Walnut Ave,None,Renton,98055,Washington,United States


In [14]:
# Step 4.5 - Validate loading of dim_territories table into database
sql_territories = "SELECT * FROM midterm_dw.dim_territories;"
df_dim_territories = get_sql_dataframe(sql_territories, **dest_db_args)
df_dim_territories.head(2)

,Territory_Key,TerritoryID,TerritoryName,CountryRegionCode,Group
0,1,1,Northwest,US,North America
1,2,2,Northeast,US,North America


Step 5 - Extract the fact dataframe via SQL

In [24]:
# Extract sales order from 'salesorderheader', 'salesorderdetail', 'creditcard', and 'shipmethod' tables
sql_fact_sales_orders = """
SELECT soh.SalesOrderID,
        soh.TerritoryID,
        soh.CustomerID,
        sod.ProductID,
		soh.OrderDate,
		soh.DueDate,
		soh.ShipDate,
		soh.Status,
		soh.OnlineOrderFlag,
		soh.SalesOrderNumber,
		soh.PurchaseOrderNumber,
		soh.SalesPersonID,
		sm.Name AS ShipMethod,
		sm.ShipBase,
		sm.ShipRate,
		cc.CardType AS `Credit Card Type`,
		cc.CardNumber AS `Credit Card Number`,
		cc.ExpMonth AS `Credit Card ExpMonth`,
		cc.ExpYear AS `Credit Card ExpYear`,
		soh.CreditCardApprovalCode,
		soh.SubTotal,
		soh.TaxAmt,
		soh.Freight,
		soh.TotalDue,
		sod.CarrierTrackingNumber,
		sod.OrderQty,
		sod.UnitPrice,
		sod.LineTotal
	FROM salesorderheader AS soh
	LEFT OUTER JOIN salesorderdetail AS sod
	ON soh.SalesOrderID = sod.SalesOrderID
	LEFT OUTER JOIN creditcard AS cc
	ON soh.CreditCardID = cc.CreditCardID
	LEFT OUTER JOIN shipmethod AS sm
	ON soh.ShipMethodID = sm.ShipMethodID
"""

df_fact_sales_orders = get_sql_dataframe(sql_fact_sales_orders, **src_db_args)
df_fact_sales_orders.head(2)


,SalesOrderID,TerritoryID,CustomerID,ProductID,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,...,Credit Card ExpYear,CreditCardApprovalCode,SubTotal,TaxAmt,Freight,TotalDue,CarrierTrackingNumber,OrderQty,UnitPrice,LineTotal
0,43659,5,676,711,2001-07-01,2001-07-13,2001-07-08,5,b'\x00',SO43659,...,2007.0,105041Vi84182,24643.9362,1971.5149,616.0984,27231.5495,4911-403C-98,4,20.1865,80.746
1,43659,5,676,712,2001-07-01,2001-07-13,2001-07-08,5,b'\x00',SO43659,...,2007.0,105041Vi84182,24643.9362,1971.5149,616.0984,27231.5495,4911-403C-98,2,5.1865,10.373


Step 6 - Transform the fact dataframe via SQL

In [25]:
# Step 6.1 - Transform the OrderDate in fact orders to serve as a key in the dim_date table
df_fact_date = df_dim_dates.rename(columns={"date_key" : "OrderDateKey", "full_date" : "OrderDate"})
df_fact_sales_orders.OrderDate = df_fact_sales_orders.OrderDate.astype('datetime64[ns]').dt.date
df_fact_sales_orders = pd.merge(df_fact_sales_orders, df_fact_date, on='OrderDate', how='left')
df_fact_sales_orders.drop(['OrderDate'], axis=1, inplace=True)
df_fact_sales_orders.head(2)

,SalesOrderID,TerritoryID,CustomerID,ProductID,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,...,CreditCardApprovalCode,SubTotal,TaxAmt,Freight,TotalDue,CarrierTrackingNumber,OrderQty,UnitPrice,LineTotal,OrderDateKey
0,43659,5,676,711,2001-07-13,2001-07-08,5,b'\x00',SO43659,PO522145787,...,105041Vi84182,24643.9362,1971.5149,616.0984,27231.5495,4911-403C-98,4,20.1865,80.746,20010701
1,43659,5,676,712,2001-07-13,2001-07-08,5,b'\x00',SO43659,PO522145787,...,105041Vi84182,24643.9362,1971.5149,616.0984,27231.5495,4911-403C-98,2,5.1865,10.373,20010701


In [26]:
# Step 6.2 - Transform the DueDate in fact orders to serve as a key in the dim_date table
df_fact_date = df_dim_dates.rename(columns={"date_key" : "DueDateKey", "full_date" : "DueDate"})
df_fact_sales_orders.DueDate = df_fact_sales_orders.DueDate.astype('datetime64[ns]').dt.date
df_fact_sales_orders = pd.merge(df_fact_sales_orders, df_fact_date, on='DueDate', how='left')
df_fact_sales_orders.drop(['DueDate'], axis=1, inplace=True)
df_fact_sales_orders.head(2)

,SalesOrderID,TerritoryID,CustomerID,ProductID,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,SalesPersonID,...,SubTotal,TaxAmt,Freight,TotalDue,CarrierTrackingNumber,OrderQty,UnitPrice,LineTotal,OrderDateKey,DueDateKey
0,43659,5,676,711,2001-07-08,5,b'\x00',SO43659,PO522145787,279.0,...,24643.9362,1971.5149,616.0984,27231.5495,4911-403C-98,4,20.1865,80.746,20010701,20010713
1,43659,5,676,712,2001-07-08,5,b'\x00',SO43659,PO522145787,279.0,...,24643.9362,1971.5149,616.0984,27231.5495,4911-403C-98,2,5.1865,10.373,20010701,20010713


In [27]:
# Step 6.3 - Transform the ShipDate in fact orders to serve as a key in the dim_date table
df_fact_date = df_dim_dates.rename(columns={"date_key" : "ShipDateKey", "full_date" : "ShipDate"})
df_fact_sales_orders.ShipDate = df_fact_sales_orders.ShipDate.astype('datetime64[ns]').dt.date
df_fact_sales_orders = pd.merge(df_fact_sales_orders, df_fact_date, on='ShipDate', how='left')
df_fact_sales_orders.drop(['ShipDate'], axis=1, inplace=True)
df_fact_sales_orders.head(2)

,SalesOrderID,TerritoryID,CustomerID,ProductID,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,SalesPersonID,ShipMethod,...,TaxAmt,Freight,TotalDue,CarrierTrackingNumber,OrderQty,UnitPrice,LineTotal,OrderDateKey,DueDateKey,ShipDateKey
0,43659,5,676,711,5,b'\x00',SO43659,PO522145787,279.0,CARGO TRANSPORT 5,...,1971.5149,616.0984,27231.5495,4911-403C-98,4,20.1865,80.746,20010701,20010713,20010708
1,43659,5,676,712,5,b'\x00',SO43659,PO522145787,279.0,CARGO TRANSPORT 5,...,1971.5149,616.0984,27231.5495,4911-403C-98,2,5.1865,10.373,20010701,20010713,20010708


In [28]:
# Step 6.4 - Use 'Product_Key' as a foreign key for the facts table to the dim_products table
sql_dim_products = "SELECT Product_Key, ProductID FROM midterm_dw.dim_products;"
df_dim_products = get_sql_dataframe(sql_dim_products, **dest_db_args)

df_fact_sales_orders = pd.merge(df_fact_sales_orders, df_dim_products, on="ProductID", how='left')
df_fact_sales_orders.drop(['ProductID'], axis=1, inplace=True)
df_fact_sales_orders.head(2)

,SalesOrderID,TerritoryID,CustomerID,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,SalesPersonID,ShipMethod,ShipBase,...,Freight,TotalDue,CarrierTrackingNumber,OrderQty,UnitPrice,LineTotal,OrderDateKey,DueDateKey,ShipDateKey,Product_Key
0,43659,5,676,5,b'\x00',SO43659,PO522145787,279.0,CARGO TRANSPORT 5,8.99,...,616.0984,27231.5495,4911-403C-98,4,20.1865,80.746,20010701,20010713,20010708,216
1,43659,5,676,5,b'\x00',SO43659,PO522145787,279.0,CARGO TRANSPORT 5,8.99,...,616.0984,27231.5495,4911-403C-98,2,5.1865,10.373,20010701,20010713,20010708,217


In [29]:
# Step 6.5 - Use 'Customer_Key' as a foreign key for the facts table to the dim_customers table
sql_dim_customers = "SELECT Customer_Key, CustomerID FROM midterm_dw.dim_customers;"
df_dim_customers = get_sql_dataframe(sql_dim_customers, **dest_db_args)

df_fact_sales_orders = pd.merge(df_fact_sales_orders, df_dim_customers, on="CustomerID", how='left')
df_fact_sales_orders.drop(['CustomerID'], axis=1, inplace=True)
df_fact_sales_orders.head(2)

,SalesOrderID,TerritoryID,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,SalesPersonID,ShipMethod,ShipBase,ShipRate,...,TotalDue,CarrierTrackingNumber,OrderQty,UnitPrice,LineTotal,OrderDateKey,DueDateKey,ShipDateKey,Product_Key,Customer_Key
0,43659,5,5,b'\x00',SO43659,PO522145787,279.0,CARGO TRANSPORT 5,8.99,1.49,...,27231.5495,4911-403C-98,4,20.1865,80.746,20010701,20010713,20010708,216,687
1,43659,5,5,b'\x00',SO43659,PO522145787,279.0,CARGO TRANSPORT 5,8.99,1.49,...,27231.5495,4911-403C-98,2,5.1865,10.373,20010701,20010713,20010708,217,687


In [30]:
# Step 6.6 - Use 'TerritoryKey' as a foreign key for the facts table to the dim_territories table
sql_dim_territories = "SELECT Territory_Key, TerritoryID FROM midterm_dw.dim_territories;"
df_dim_territories = get_sql_dataframe(sql_dim_territories, **dest_db_args)

df_fact_sales_orders = pd.merge(df_fact_sales_orders, df_dim_territories, on="TerritoryID", how='left')
df_fact_sales_orders.drop(['TerritoryID'], axis=1, inplace=True)
df_fact_sales_orders.head(2)

,SalesOrderID,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,SalesPersonID,ShipMethod,ShipBase,ShipRate,Credit Card Type,...,CarrierTrackingNumber,OrderQty,UnitPrice,LineTotal,OrderDateKey,DueDateKey,ShipDateKey,Product_Key,Customer_Key,Territory_Key
0,43659,5,b'\x00',SO43659,PO522145787,279.0,CARGO TRANSPORT 5,8.99,1.49,ColonialVoice,...,4911-403C-98,4,20.1865,80.746,20010701,20010713,20010708,216,687,5
1,43659,5,b'\x00',SO43659,PO522145787,279.0,CARGO TRANSPORT 5,8.99,1.49,ColonialVoice,...,4911-403C-98,2,5.1865,10.373,20010701,20010713,20010708,217,687,5


In [32]:
# Step 6.7 - Perform final transformations of df_fact_sales_orders

# Drop any unneeded/not well formatted columns
df_fact_sales_orders.drop(['OnlineOrderFlag', 'SalesPersonID'], axis=1, inplace=True)

# Add new primary key to fact_sales_orders
df_fact_sales_orders.insert(0, "FactSalesOrder_Key", range(1, df_fact_sales_orders.shape[0]+1))

# Rearrange columns in facts table
ordered_columns = [
    'FactSalesOrder_Key', 'Product_Key', 'Customer_Key', 'Territory_Key', 'OrderDateKey', 'ShipDateKey', 'DueDateKey',
    'SalesOrderID', 'Status', 'SalesOrderNumber', 'PurchaseOrderNumber', 'ShipMethod', 'ShipBase', 'ShipRate',
    'Credit Card Type', 'Credit Card Number', 'Credit Card ExpMonth', 'Credit Card ExpYear', 'CreditCardApprovalCode',
    'SubTotal', 'TaxAmt', 'Freight', 'TotalDue', 'CarrierTrackingNumber', 'OrderQty', 'UnitPrice', 'LineTotal'
]
df_fact_sales_orders = df_fact_sales_orders[ordered_columns]

df_fact_sales_orders.head(2)

,FactSalesOrder_Key,Product_Key,Customer_Key,Territory_Key,OrderDateKey,ShipDateKey,DueDateKey,SalesOrderID,Status,SalesOrderNumber,...,Credit Card ExpYear,CreditCardApprovalCode,SubTotal,TaxAmt,Freight,TotalDue,CarrierTrackingNumber,OrderQty,UnitPrice,LineTotal
0,1,216,687,5,20010701,20010708,20010713,43659,5,SO43659,...,2007.0,105041Vi84182,24643.9362,1971.5149,616.0984,27231.5495,4911-403C-98,4,20.1865,80.746
1,2,217,687,5,20010701,20010708,20010713,43659,5,SO43659,...,2007.0,105041Vi84182,24643.9362,1971.5149,616.0984,27231.5495,4911-403C-98,2,5.1865,10.373


Step 7 - Load the fact dataframe into the new database

In [33]:
# Step 7.1 - Load into database
set_dataframe(df_fact_sales_orders, "fact_sales_orders", "FactSalesOrder_Key", "insert", **dest_db_args)

In [34]:
# Step 7.2 - Ensure database contains the data
sql_fact_sales_orders = "SELECT * FROM midterm_dw.fact_sales_orders;"
df_fact_sales_orders = get_sql_dataframe(sql_fact_sales_orders, **dest_db_args)
df_fact_sales_orders.head(2)

,FactSalesOrder_Key,Product_Key,Customer_Key,Territory_Key,OrderDateKey,ShipDateKey,DueDateKey,SalesOrderID,Status,SalesOrderNumber,...,Credit Card ExpYear,CreditCardApprovalCode,SubTotal,TaxAmt,Freight,TotalDue,CarrierTrackingNumber,OrderQty,UnitPrice,LineTotal
0,1,216,687,5,20010701,20010708,20010713,43659,5,SO43659,...,2007.0,105041Vi84182,24643.9362,1971.5149,616.0984,27231.5495,4911-403C-98,4,20.1865,80.746
1,2,217,687,5,20010701,20010708,20010713,43659,5,SO43659,...,2007.0,105041Vi84182,24643.9362,1971.5149,616.0984,27231.5495,4911-403C-98,2,5.1865,10.373


Step 8 - Demonstrate proper functionality in new database

To demonstrate proper functionality of my data warehouse, I will perform the following two SQL SELECT queries:
1. The first query should return the total number of sales for each territory of a product by the name: "Sport-100 Helmet, Red".
2. The second query should return the first and last name of the top 10 customers that have spent the most in the US territory. If the data warehouse does not contain a first and last name for the customer, do not include this value (as not all customers have first and last name data from the *adventureworks* database).

In [37]:
# Query 1
sql_query_one = """
    SELECT t.TerritoryName,
        Count(*),
        p.Name
    FROM fact_sales_orders as f
    JOIN dim_products AS p
    ON f.Product_Key = p.Product_Key
    JOIN dim_territories AS t
    ON f.Territory_Key = t.Territory_Key
    WHERE p.Name = 'Sport-100 Helmet, Red'
    GROUP BY t.TerritoryName; 
"""
df_query_one = get_sql_dataframe(sql_query_one, **dest_db_args)
df_query_one.head(20)

,TerritoryName,Count(*),Name
0,Northwest,392,"Sport-100 Helmet, Red"
1,Canada,458,"Sport-100 Helmet, Red"
2,Northeast,78,"Sport-100 Helmet, Red"
3,Southwest,624,"Sport-100 Helmet, Red"
4,Southeast,93,"Sport-100 Helmet, Red"
5,Central,89,"Sport-100 Helmet, Red"
6,United Kingdom,336,"Sport-100 Helmet, Red"
7,Australia,467,"Sport-100 Helmet, Red"
8,France,290,"Sport-100 Helmet, Red"
9,Germany,280,"Sport-100 Helmet, Red"


In [38]:
sql_query_two = """
    SELECT 
        c.FirstName,
        c.LastName,
        SUM(f.TotalDue) AS TotalSpent
    FROM fact_sales_orders as f
    JOIN dim_customers AS c
    ON f.Customer_Key = c.Customer_Key
    JOIN dim_territories AS t
    ON f.Territory_Key = t.Territory_Key
    WHERE t.CountryRegionCode = 'US' 
        AND c.FirstName IS NOT NULL 
        AND c.LastName IS NOT NULL
    GROUP BY c.CustomerID, c.FirstName, c.LastName
    ORDER BY TotalSpent DESC
    LIMIT 10;
"""
df_query_two = get_sql_dataframe(sql_query_two, **dest_db_args)
df_query_two.head(10)

,FirstName,LastName,TotalSpent
0,José,Miller,22901.9432
1,Nathaniel,Cooper,22749.4973
2,Miguel,Sanchez,22718.5573
3,Jason,Mitchell,22571.5923
4,Cheryl,Dominguez,21524.6928
5,Gilbert,Xu,20661.4672
6,Barry,Subram,20514.4466
7,Elijah,Ross,20303.1046
8,Isabel,Jenkins,19978.2346
9,Noah,Scott,19809.1696
